In [1]:
import requests
import pandas as pd
import json

/Users/jose/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# ----------------------------
# League IDs & roster mapping
# ----------------------------
LEAGUE_IDS = {
    2024: "1064047033306136576",
    2025: "1180226065723957248"
}

ROSTER_TO_OWNER = {
    1: "Jose",
    2: "Daryl",
    3: "Gio",
    4: "Brigido",
    5: "Luis",
    6: "Lalo",
    7: "Devonte",
    8: "Marvin",
    9: "Jacky",
    10: "Bryan"
}

In [3]:
# ============================
# 2️⃣ PULL PLAYER DATA
# ============================
players_url = "https://api.sleeper.app/v1/players/nfl"
players_data = requests.get(players_url).json()

# Keep only relevant info
players_info = {pid: {"position": pdata.get("position"), "name": pdata.get("full_name")} 
                for pid, pdata in players_data.items()}


In [4]:
# ============================
# 3️⃣ PULL MATCHUPS
# ============================
def pull_matchups(league_id, weeks=range(1, 14)):
    all_matchups = {}
    for week in weeks:
        url = f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week}"
        response = requests.get(url)
        if response.status_code == 200:
            all_matchups[f"week_{week}"] = response.json()
        else:
            print(f"Failed to load week {week}")
    return all_matchups

matchups_data = {
    year: pull_matchups(league_id) for year, league_id in LEAGUE_IDS.items()
}

# Save locally
with open("matchups_2024_2025.json", "w") as f:
    json.dump(matchups_data, f, indent=4)

In [5]:
# ============================
# 4️⃣ AGGREGATE WEEKLY FEATURES
# ============================
def aggregate_weekly_features(matchups, players_info):
    rows = []
    for week, week_data in matchups.items():
        for m in week_data:
            roster_id = m['roster_id']
            owner_name = m.get('owner_name', ROSTER_TO_OWNER.get(roster_id, str(roster_id)))
            starters = m.get('starters', [])
            
            # Initialize points and counts
            starter_points = {'QB':0, 'RB':0, 'WR':0, 'TE':0}
            starter_counts = {'QB':0, 'RB':0, 'WR':0, 'TE':0}
            
            for pid in starters:
                pid_str = str(pid)
                if pid_str in players_info:
                    pos = players_info[pid_str]['position']
                    pts = m.get('points', 0)  # can be replaced with actual player weekly points if available
                    if pos in starter_points:
                        # Tiered scoring
                        if pos == 'RB':
                            pts *= 0.5
                        elif pos == 'WR':
                            pts *= 1
                        elif pos == 'TE':
                            pts *= 1.5
                        starter_points[pos] += pts
                        starter_counts[pos] += 1
            
            rows.append({
                'week': int(week.split('_')[1]),
                'owner': owner_name,
                'roster_id': roster_id,
                'QB_pts': starter_points['QB'],
                'RB_pts': starter_points['RB'],
                'WR_pts': starter_points['WR'],
                'TE_pts': starter_points['TE'],
                'QB_count': starter_counts['QB'],
                'RB_count': starter_counts['RB'],
                'WR_count': starter_counts['WR'],
                'TE_count': starter_counts['TE'],
                'total_points': m.get('points', 0)
            })
    return pd.DataFrame(rows)

In [6]:
# ============================
# 5️⃣ COMBINE 2024 & 2025 DATA
# ============================
df_2024 = aggregate_weekly_features(matchups_data[2024], players_info)
df_2025 = aggregate_weekly_features(matchups_data[2025], players_info)

df_2024['year'] = 2024
df_2025['year'] = 2025
df_2024['owner'] = df_2024['roster_id'].map(ROSTER_TO_OWNER)
df_2025['owner'] = df_2025['roster_id'].map(ROSTER_TO_OWNER)

combined_df = pd.concat([df_2024, df_2025], ignore_index=True)

In [7]:
combined_df.head()

,week,owner,roster_id,QB_pts,RB_pts,WR_pts,TE_pts,QB_count,RB_count,WR_count,TE_count,total_points,year
0,1,Jose,1,239.08,119.54,478.16,358.62,2,2,4,2,119.54,2024
1,1,Daryl,2,258.88,129.44,647.20,194.16,2,2,5,1,129.44,2024
2,1,Gio,3,278.00,139.00,695.00,208.50,2,2,5,1,139.00,2024
3,1,Brigido,4,317.80,238.35,635.60,238.35,2,3,4,1,158.90,2024
4,1,Luis,5,234.04,175.53,468.08,175.53,2,3,4,1,117.02,2024


In [8]:
picks = pd.read_parquet('../Data/all_pick_counts.parquet').drop_duplicates()

In [9]:
import numpy as np
import pandas as pd

# ============================
# 6️⃣ BUILD FUTURE CAPITAL SCORE
# ============================

# Only future rookie capital matters for projection
future_picks = picks[picks['year'] == 2026].copy()

# Weight closer years more heavily
#YEAR_WEIGHTS = {
#    2026: 0.6,
#    2027: 0.4
#}

#future_picks['year_weight'] = future_picks['year'].map(YEAR_WEIGHTS)

# Weighted pick score
future_capital = (
    future_picks
    .groupby('owner_name', as_index=False)['pick_score_norm']
    .sum()
    .rename(columns={'pick_score_norm': 'future_capital_score'})
)

# Normalize 0–1
future_capital['future_capital_score'] = (
    (future_capital['future_capital_score'] - future_capital['future_capital_score'].min()) /
    (future_capital['future_capital_score'].max() - future_capital['future_capital_score'].min())
)

# ============================
# 7️⃣ AGGREGATE REAL 2025 SCORING
# ============================

avg_2025 = (
    combined_df[combined_df['year'] == 2025]
    .groupby('owner', as_index=False)['total_points']
    .mean()
    .rename(columns={
        'owner': 'owner_name',
        'total_points': 'avg_points_2025_real'
    })
)

# ============================
# 8️⃣ MERGE MODEL INPUTS
# ============================

model_df = (
    avg_2025
    .merge(future_capital, on='owner_name', how='left')
    .fillna({'future_capital_score': 0})
)

# ============================
# 9️⃣ DYNASTY REGRESSION LOGIC
# ============================

BASE_REGRESSION = 0.06   # Natural year-over-year decline
DRAFT_OFFSET = 0.10     # Draft capital can offset decline

model_df['draft_offset'] = model_df['future_capital_score'] * DRAFT_OFFSET
model_df['net_growth_rate'] = -BASE_REGRESSION + model_df['draft_offset']

# ============================
# 🔮 PROJECT 2026 SCORING
# ============================

model_df['projected_avg_points_2026'] = (
    model_df['avg_points_2025_real'] *
    (1 + model_df['net_growth_rate'])
)

model_df['projected_weekly_change_2026'] = (
    model_df['projected_avg_points_2026'] -
    model_df['avg_points_2025_real']
)

# ============================
# 🏆 COMPOSITE POWER SCORE (CPS)
# ============================

# Normalize current scoring
model_df['avg_points_norm'] = (
    (model_df['avg_points_2025_real'] - model_df['avg_points_2025_real'].min()) /
    (model_df['avg_points_2025_real'].max() - model_df['avg_points_2025_real'].min())
)

# CPS weighting: 70% roster strength, 30% future capital
model_df['cps'] = (
    0.7 * model_df['avg_points_norm'] +
    0.3 * model_df['future_capital_score']
)

# ============================
# 🔟 FINAL OUTPUT
# ============================

final_projection = (
    model_df
    .sort_values('projected_avg_points_2026', ascending=False)
    .reset_index(drop=True)
)

final_projection[
    [
        'owner_name',
        'avg_points_2025_real',
        'projected_avg_points_2026',
        'projected_weekly_change_2026',
        'future_capital_score',
        'cps'
    ]
]


,owner_name,avg_points_2025_real,projected_avg_points_2026,projected_weekly_change_2026,future_capital_score,cps
0,Devonte,175.503077,164.972892,-10.530185,0.000000,0.700000
1,Lalo,149.958462,155.956800,5.998338,1.000000,0.603885
2,Luis,153.189231,148.911494,-4.277737,0.320755,0.450210
3,Bryan,146.236923,147.395782,1.158859,0.679245,0.449950
4,Daryl,154.089231,144.843877,-9.245354,0.000000,0.367940
5,Jacky,147.478462,138.629754,-8.848708,0.000000,0.265428
6,Gio,136.040000,134.807940,-1.232060,0.509434,0.240885
7,Jose,130.760000,132.536362,1.776362,0.735849,0.226934
8,Marvin,130.361538,131.148627,0.787089,0.660377,0.198113
9,Brigido,132.732308,125.519684,-7.212624,0.056604,0.053744
